# Covid-19 Time Series Modeling

- Flex
- Instructor: Morgan Jones
- Blog: [https://datascienceprojectsandmore.blogspot.com/](https://datascienceprojectsandmore.blogspot.com/)
- Date of Review: Tuesday, April 9th 2024

# Abstract

This project creates a model that forecasts the number of deaths by Covid-19 given the historical data.  The Covid-19 pandemic was detrimental in the number of people who perished from this disease.  The idea behind this project is that if we can create a predictive model that will forecast the amount of harm, then public health officials can be better informed, advised, and empowered to mitigate future deaths.  Specifically, this project focuses on a continent-wide evaluation, looking at Asia.

This project goes through a model iteration process, starting with a naive time series model, and iterating through ARIMA, multivariate ARIMA, Linear Regression, Facebook's Prophet, and multivariate prophet models.  The project finalizes on the multivariate prophet model as the final model based on the evaluation metric, root mean squared error, and evaluates this model on the holdout test set.  Lastly, the project offers a few recommendations to the proposed business and next steps for future projects.

- [GitHub README]()
- [non-technical presentation]()

# Introduction

## The Covid-19 Pandemic

![covid_map](images/covid_map.jpg)

The Covid-19 pandemic needs little introduction as it left no corner of our world untouched.  It devastated lives, and economies -- day-to-day life was dramatically altered for about 2-3 years.  During the pandemic, the World Health Organization, along with many other public health entities, concerned themselves with not only reacting to this public health crisis but also to recording and analyzing the data from the outbreak. This data accumulation was not only a way to evaluate the devastation and how different countries handled the situation, but also to allowed them to provide better recommendations and make impactful decisions concerning public health. 

##  This Project | The Business + Business Problem

The specific business for this project is one akin to the World Health Organization (WHO) that 'treats data as a public good'([WHO Principles](https://data.who.int/about/data/who-data-principles), and also wants to leverage this data to make recommendations to public health officials.  When the pandemic arose, the WHO aided in the response including "facilitating research, develping guidance, coordinating vaccine development and distribution, and monitoring daily case numbers and trends around the world" ([WHO_COVID-19](https://www.who.int/news-room/fact-sheets/detail/coronavirus-disease-(covid-19)). Another company, Our World in Data, has extensive documentation of the Covid-19 pandemic ([Our World in Data/coronavirus](https://ourworldindata.org/coronavirus)).

The idea for this project is to create a model that will forecast how much worse the pandemic will be in order to advise public health officials and policy makers on how to best handle the situation such that damages are minimized. We will use the number of deaths as the target to forecast and evaluate how harmful the outbreak is (or will be for predictions).

## The Data Source

The Data from this project is sourced directly from Kaggle, a dataset called [Our World in Data - COVID-19](https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset).  The company, [Our World in Data](https://ourworldindata.org/explorers/coronavirus-data-explorer) combined their own data along with data from the John Hopkins University and the WHO.  Further details on their data and sources can be found on the [GitHub page here](https://github.com/owid/covid-19-data/blob/master/public/data/README.md).

For visual purposes, the project also utilizes a dataset of latitude and longitude data of countries from Kaggle, originally sourced from google.  See that data set [here](https://www.kaggle.com/datasets/paultimothymooney/latitude-and-longitude-for-every-country-and-state).

### Reproducing via Google CoLabs

If you are reproducing this project via Google CoLabs, please uncomment the two `!pip install` commands in the imports cell below, along with the 3 cells of code below the versions print outs.  The pip install commands will allow the use of pmdarima and Prophet libraries.  The three cells to uncomment will configure the Google CoLab environment to download the data sources from Kaggle, however **be sure to enter your own Kaggle username and API**, otherwise this cell will not work.  After configuring the cells download the two datasets and unzip them for further use.

In [1]:
# Import Needed Libraries

import json
import shutil
import pandas as pd
import numpy as np
import scipy

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('dark')

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.regression.linear_model import OLS

# For Google CoLab might need to install pmdarima and Prophet
#!pip install pmdarima
#!pip install Prophet

from pmdarima.arima import auto_arima
from prophet import Prophet
from prophet.utilities import regressor_coefficients

In [2]:
# Versions
import sys
import matplotlib
import sklearn
import statsmodels as sm
import prophet
import pmdarima

print('Python | ', sys.version[:5])
print('Pandas | ', pd.__version__)
print('Numpy | ', np.__version__)
print('SciPy | ', scipy.__version__)
print('Matplotlib | ', matplotlib.__version__)
print('Seaborn | ', sns.__version__)
print('Geopandas | ', gpd.__version__)
print('SciKit Learn | ', sklearn.__version__)
print('Stats Models | ', sm.__version__)
print('pmdarima | ', pmdarima.__version__)
print('Prophet | ', prophet.__version__)

Python |  3.8.5
Pandas |  1.1.3
Numpy |  1.24.4
SciPy |  1.10.1
Matplotlib |  3.3.1
Seaborn |  0.11.0
Geopandas |  0.13.2
SciKit Learn |  1.3.2
Stats Models |  0.14.1
pmdarima |  2.0.4
Prophet |  1.1.5


In [3]:
# Prep Google CoLab environment to download data from Kaggle
#!mkdir ~/.kaggle
#!touch ~/.kaggle/kaggle.json

#username = ''  ## Your Kaggle username
#api_key = ''  ## Your Kaggle API key

#api_token = {"username": username,
#             "key": api_key}

#with open('/root/.kaggle/kaggle.json', 'w') as file:
#    json.dump(api_token, file)

#!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset from Kaggle
#!kaggle datasets download -d caesarmario/our-world-in-data-covid19-dataset
#!kaggle datasets download -d paultimothymooney/latitude-and-longitude-for-every-country-and-state

In [5]:
# This cell unzips the downloaded data
#shutil.unpack_archive('our-world-in-data-covid19-dataset.zip', '/content')
#shutil.unpack_archive('latitude-and-longitude-for-every-country-and-state.zip', '/content')

# Data Exploration

First, we look at the complete data and find that there are quite a few redundancies in the accumulated data.  With the idea of modeling in mind, many of the columns are dropped.  For example, the data includes `total_cases`, `new_cases`, `new_cases_smoothed`, `new_cases_per_million`, and `new_cases_smoothed_per_million`.  For modeling, specifically for time series modeling, we are interested in the daily reports, and thus keep `new_cases_smoothed` because (a) smoothed data is better for seeing trends and (b) we want to work with the real data of the number of new cases that occurred each day.  This logic is applied to many of the variables in the original data set.  See below for a list of the variables kept and their definitions.  See also the [GitHub page](https://github.com/owid/covid-19-data/blob/master/public/data/README.md) for a full list of all the variables and their descriptions. 

Next, a visual of the countries in which the data is coming from is produced. The idea behind this is to try and see the best way to splice the data -- is it best by individual country, continent or by something else?  After looking at this map, it is decided to break up the data by continent and focus on Asia.  *(The original idea was to create models for each continent, but this project only had the bandwidth to look at one.  Future work would be to expand this to the other continents and compare.)* 

The project then transitions into a data cleaning phase in order to deal with the missing values in the Asia subset before looking at other 'data exploration' visuals.  The reasoning behind this is we want to explore the dataset that will be used in modeling; however this needs to be created by aggregating the Asia subset such that there is only one entry per day.  The missing values must be dealt with prior to aggregating.  See below for further details that occur during the Data Cleaning phase. 

> One note concerning the data cleaning phase.  A limitation of this project in specific is not only the amount of missing data in the raw data source, but also the question behind the validity of the data itself.  While the WHO is a very reputable source and is to be trusted, it is widely known that countries in Asia, namely China and India, were suspected of purposefully mis-reporting deaths due to Covid-19.  It is important to keep in mind that the model can only ever be as good as the data used.  (see [China is 'heavily underreporting' ...](https://abcnews.go.com/Health/china-heavily-underreporting-number-covid-19-deaths/story?id=96389383), [China deletes Covid-19 death data](https://www.ft.com/content/a634d844-5298-441b-b2e8-0eabe0b3c1d7), [WHO says China revised...](https://www.cnbc.com/2020/04/17/who-says-china-revised-coronavirus-infection-data-to-leave-no-case-undocumented.html), [COVID mortality in India](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9836201/), and [WHO says millions of Covid deaths...](https://timesofindia.indiatimes.com/india/who-says-millions-of-covid-deaths-went-unreported-in-india-centre-strongly-objects-methodology-key-points/articleshow/91349479.cms) for further reading).

Lastly, the project goes over some visuals for the aggregated Asia subset data that will be used for modeling, including showing the target variable that will be used for the strictly time series models (as well as all the models), the nubmer of new deaths per day.

In [6]:
# Look at data
df = pd.read_csv('owid-covid-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350085 entries, 0 to 350084
Data columns (total 67 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   iso_code                                    350085 non-null  object 
 1   continent                                   333420 non-null  object 
 2   location                                    350085 non-null  object 
 3   date                                        350085 non-null  object 
 4   total_cases                                 312088 non-null  float64
 5   new_cases                                   340457 non-null  float64
 6   new_cases_smoothed                          339198 non-null  float64
 7   total_deaths                                290501 non-null  float64
 8   new_deaths                                  340511 non-null  float64
 9   new_deaths_smoothed                         339281 non-null  float64
 


| Columns | Description | 
|:--|:--| 
| `continent` | Continent of the geographical location | 
| `location` | Geographical location | 
| `date` | Date of observation | 
| `new_cases` | New confirmed cases of COVID-19. Counts can include probable cases, where reported. In rare cases where our source reports a negative daily change due to a data correction, we set this metric to NA | 
| `new_cases_smoothed` | New confirmed cases of COVID-19 (7-day smoothed). Counts can include probable cases, where reported. | 
| `new_deaths` | New deaths attributed to COVID-19. Counts can include probable deaths, where reported. In rare cases where our source reports a negative daily change due to a data correction, we set this metric to NA. |
| `new_deaths_smoothed` | New deaths attributed to COVID-19 (7-day smoothed). Counts can include probable deaths, where reported. |
| `reproduction_rate` | Real-time estimate of the effective reproduction rate (R) of COVID-19. See https://github.com/crondonm/TrackingR/tree/main/Estimates-Database | 
| `icu_patients` | Number of COVID-19 patients in intensive care units (ICUs) on a given day. |
| `hosp_patients` | Number of COVID-19 patients in hospital on a given day. |
| `new_tests` | New tests for COVID-19 (only calculated for consecutive days). |
| `new_tests_smoothed` | New tests for COVID-19 (7-day smoothed). For countries that don't report testing data on a daily basis, we assume that testing changed equally on a daily basis over any periods in which no data was reported. This produces a complete series of daily figures, which is then averaged over a rolling 7-day window |
| `positive_rate` | The share of COVID-19 tests that are positive, given as a rolling 7-day average (this is the inverse of tests_per_case). |
| `test_units` | Units used by the location to report its testing data. A country file can't contain mixed units. All metrics concerning testing data use the specified test unit. Valid units are 'people tested' (number of people tested), 'tests performed' (number of tests performed. a single person can be tested more than once in a given day) and 'samples tested' (number of samples tested. In some cases, more than one sample may be required to perform a given test.) | 
| `people_vaccinated` | Total number of people who received at least one vaccine dose | 
| `new_people_vaccinated_smoothed` | Daily number of people receiving their first vaccine dose (7-day smoothed) | 
| `new_vaccinations` | New COVID-19 vaccination doses administered (only calculated for consecutive days) | 
| `new_vaccinations_smoothed` | New COVID-19 vaccination doses administered (7-day smoothed). For countries that don't report vaccination data on a daily basis, we assume that vaccination changed equally on a daily basis over any periods in which no data was reported. This produces a complete series of daily figures, which is then averaged over a rolling 7-day window |
| `total_boosters` | Total number of COVID-19 vaccination booster doses administered (doses administered beyond the number prescribed by the vaccination protocol) |
| `stringency_index` | Government Response Stringency Index: composite measure based on 9 response indicators including school closures, workplace closures, and travel bans, rescaled to a value from 0 to 100 (100 = strictest response). | 
| `population_density` | Number of people divided by land area, measured in square kilometers, most recent year available | 
| `median_age` | Median age of the population, UN projection for 2020 | 
| `aged_70_older` | Share of the population that is 70 years and older in 2015 |
| `gdp_per_capita` | Gross domestic product at purchasing power parity (constant 2011 international dollars), most recent year available | 
| `extreme_poverty` | Share of the population living in extreme poverty, most recent year available since 2010 | 
| `cardiovasc_death_rate` | Death rate from cardiovascular disease in 2017 (annual number of deaths per 100,000 people) | 
| `diabetes_prevalence` | Dabetes prevalence (% of population aged 20 to 79) in 2017 |
| `female_smokers` | Share of women who smoke, most recent year available | 
| `male_smokers` | Share of men who smoke, most recent year available | 
| `handwashing_facilities` | Share of the population with basic handwashing facilities on premises, most recent year available |
| `hospital_beds_per_thousand` | Hospital beds per 1,000 people, most recent year available since 2010 |
| `life_expectancy` | Life expectancy at birth in 2019 | 
| `human_development_index` | A composite index measuring average achievement in three basic dimensions of human development—a long and healthy life, knowledge and a decent standard of living. Values for 2019, imported from http://hdr.undp.org/en/indicators/137506 | 
| `population` | Population (latest available values). See https://github.com/owid/covid-19-data/blob/master/scripts/input/un/population_latest.csv for full list of sources | 



In [7]:
# Choose which columns to use/drop
cols_to_use = ['continent', 'location', 'date', 'new_cases_smoothed', 'new_deaths_smoothed',
               'reproduction_rate', 'icu_patients', 'hosp_patients', 'new_tests_smoothed', 
               'positive_rate', 'tests_units', 'people_vaccinated', 'new_people_vaccinated_smoothed',
               'total_boosters', 'new_vaccinations_smoothed', 'stringency_index', 'population_density', 
               'median_age', 'aged_70_older', 'gdp_per_capita', 'extreme_poverty', 'cardiovasc_death_rate',
               'diabetes_prevalence', 'female_smokers', 'male_smokers', 'handwashing_facilities', 
               'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'population']
df1 = df[cols_to_use]

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350085 entries, 0 to 350084
Data columns (total 30 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   continent                       333420 non-null  object 
 1   location                        350085 non-null  object 
 2   date                            350085 non-null  object 
 3   new_cases_smoothed              339198 non-null  float64
 4   new_deaths_smoothed             339281 non-null  float64
 5   reproduction_rate               184817 non-null  float64
 6   icu_patients                    37615 non-null   float64
 7   hosp_patients                   38902 non-null   float64
 8   new_tests_smoothed              103965 non-null  float64
 9   positive_rate                   95927 non-null   float64
 10  tests_units                     106788 non-null  object 
 11  people_vaccinated               75911 non-null   float64
 12  new_people_vacci

In [9]:
# Update date variable to datetime
df1['date'] = pd.to_datetime(df1['date'])

<ipython-input-9-ff5bda607d92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['date'] = pd.to_datetime(df1['date'])


In [10]:
# Import latitude & longitude data
lat_long = pd.read_csv('world_country_and_usa_states_latitude_and_longitude_values.csv')
lat_long = lat_long[['latitude', 'longitude', 'country']]
lat_long.head()

,latitude,longitude,country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [11]:
# merge dataframes so can visualize
df2 = pd.merge(df1,
              lat_long,
              left_on='location',
              right_on='country',
              how='left')

In [12]:
# Drop not countries
loc_to_drop = ['Africa', 'Asia', 'Europe', 'European Union', 'High income', 
               'Low income', 'Lower middle income', 'North America', 'Oceania', 
               'South America', 'Upper middle income', 'World']

df2 = df2[~df2['location'].isin(loc_to_drop)]

In [13]:
# Visualize where, which countries, the data is coming from
    # See https://stackoverflow.com/a/53233489 for source

geometry = [Point(xy) for xy in zip(df2['longitude'], df2['latitude'])]
gdf = GeoDataFrame(df2[['longitude', 'latitude']], geometry=geometry)   

# A simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

KeyboardInterrupt: 

Based on the above graph on where the data is coming from -- the most logical and least inhibited route will be to consolidate the data by continent. 

In [ ]:
# Drop non-countries from df1
df1 = df1[~df1['location'].isin(loc_to_drop)]

In [ ]:
# sanity check
df1['continent'].isna().sum()

In [ ]:
# What are all the continents present
df1['continent'].unique()

In [ ]:
# Create the different dataframes that we will use for modeling

df_asia = df1[df1['continent'] == 'Asia']
df_eu = df1[df1['continent'] == 'Europe']
df_af = df1[df1['continent'] == 'Africa']
df_oc = df1[df1['continent'] == 'Oceania']
df_na = df1[df1['continent'] == 'North America']

df_sa = df1[df1['continent'] == 'South America']

In [ ]:
# Look at the total number of missing values by continent

data = [('Asia', df_asia), ('Europe', df_eu), ('Africa', df_af), 
        ('Oceana', df_oc), ('North America', df_na), ('South America', df_sa)]

for (name, df) in data:
    nulls = df.isna().sum().sum()
    print(name, nulls)

In [ ]:
# Visualize the countries working with in the Asia subset

df2_asia = df2[df2['continent'] == 'Asia']

geometry = [Point(xy) for xy in zip(df2_asia['longitude'], df2_asia['latitude'])]
gdf = GeoDataFrame(df2_asia[['longitude', 'latitude']], geometry=geometry)

asia = world[world['continent'] == 'Asia']
gdf.plot(ax=asia.plot(figsize=(10, 6)), marker='o', color='red', markersize=15);

## Data Cleaning 

In the below section the project handles the missing values. We first take a look at what percent of each variable is missing as well as what percent of each country is missing before looking at a visual of the number of missing values by variable by date.  This, along with the context of each variable, gives us an idea of how to appropriate deal with the missing values.  See below for the details 

#### Train/Validate/Test Split? 

The true train, validate, test split will occur after we aggregate the data to one row per day entry.  However, we do want to keep this split in mind to avoid any unnecessary data leakage.     

In [ ]:
# Look at dates where train, validate, test split will be

unique_dates_total = len(df_asia['date'].unique())
first_80_perc = unique_dates_total * .8
train_end = df_asia['date'].unique()[int(first_80_perc)]
print('Last train date:', train_end)

val_perc = first_80_perc + (unique_dates_total * .1)
val_end = df_asia['date'].unique()[int(val_perc)]
print('Last validation date:', val_end)

In [ ]:
# Check percent missing by variable
num_rows = len(df_asia['date'])
for var in df_asia.columns:
    missing = df_asia[var].isna().sum()
    if missing > 0:
        print(var, '{}%'.format(round(missing/num_rows*100, 1)))

In [ ]:
# Check percent missing by country
for country in df_asia['location'].unique():
    mask = df_asia['location'] == country
    num_cells = len(df_asia[mask]) * 30
    missing = df_asia[mask].isna().sum().sum()
    percent = round(missing/num_cells*100, 1)
    if percent > 40:
        print(country, '{}%'.format(percent))

In [ ]:
# Visualize number of missing values by variable by date

fig, axes = plt.subplots(ncols=6, nrows=5, figsize=(20,10), sharey=True, sharex=True)

for i,var in enumerate(df_asia.columns):
    y = []
    for date in df_asia['date'].unique():
        y.append(df_asia[df_asia['date'] == date][var].isna().sum())
    
    row = i // 6
    col = i % 6
    ax = axes[row, col]   
    
    ax.plot(df_asia['date'].unique(), y)
    ax.set_title(var)
    ax.set_xticks([])
plt.suptitle('Number of Nulls by Variable by Date', fontsize='xx-large');

Based on the above information, we will move forward with:
- Dropping the below as too much data is missing:
    - Country: Northern Cyrus, Macao, and Hong Kong (while not quite 50% missing, Hong Kong is missing quite a lot and is a small regional area -- can be lumped in with China). 
    - Variables: `icu_patients`, `hosp_patients`, `new_tests_smoothed`, `positive_rate`, `tests_units`, `total_boosters`, 
- Furthermore Dropping:
    - `reproduction_rate` -- as the value cannot be appropriately estimated in the validation and test sets when a vast majority of the countries stopped reporting on this value.
    - `new_people_vaccinated_smoothed` -- as we will use new vaccinations smoothed instead.
- Drop the below as they will be constant for every day (thus not useful for time series modeling):
    - `population_density`, `median_age`, `aged_70_older`, `gdp_per_capita`, `extreme_poverty`, `cardiovasc_death_rate`, `diabetes_prevalence`, `female_smokers`, `male_smokers`, `handwashing_facilities`, `hospital_beds_per_thousand`, `life_expectancy`, `human_development`

In [ ]:
# Drop countries
mask = (df_asia['location'] == 'Northern Cyprus') | (df_asia['location'] == 'Hong Kong') | (df_asia['location'] == 'Macao')
df_asia = df_asia[~mask]
    
# Drop variables
col_to_drop = ['icu_patients', 'hosp_patients', 'new_tests_smoothed', 'positive_rate', 'tests_units', 'reproduction_rate',
              'new_people_vaccinated_smoothed', 'total_boosters', 'extreme_poverty', 'handwashing_facilities',
              'population_density', 'gdp_per_capita', 'diabetes_prevalence', 'female_smokers', 'male_smokers', 
              'hospital_beds_per_thousand', 'human_development_index']

df_asia = df_asia.drop(col_to_drop, axis=1)

In [ ]:
# Recheck missing values be variable
num_rows = len(df_asia['date'])
for var in df_asia.columns:
    missing = df_asia[var].isna().sum()
    if missing > 0:
        print(var, '{}%'.format(round(missing/num_rows*100, 1)))

In [ ]:
# Recheck missing values by country
for country in df_asia['location'].unique():
    mask = df_asia['location'] == country
    num_cells = len(df_asia[mask]) * 30
    missing = df_asia[mask].isna().sum().sum()
    percent = round(missing/num_cells*100, 1)
    if percent > 20:
        print(country, '{}%'.format(percent))

| Variable | How to Handel missing values | 
|:---|:--|
| `new_cases_smoothed` | Input 0.  NaN only entered when negative values present due to corrective reporting. | 
| `new_deaths_smoothed` | Input 0. NaN only entered when negative values present due to corrective reporting. | 
| `people_vaccinated` | Forward fill for missing values after vaccine released by country. | 
| `new_vaccinations_smoothed` | Input 0. Missing values prior to vaccine release date are 0. Assume no new vaccinations or reporting error for NaN values after vaccine date. | 
| `stringency_index` | By country, forward fill. Assuming a missing value means it was not reported and no change between previous assigned value. For validation & test sets with no information to forward, assume 0 value (many Covid-19 policies ended in 2023).| 

In [ ]:
# Input 0 for assigned columns
    # True for train, validate, and test groups -- no need to split
fill_values = {'new_cases_smoothed' : 0, 'new_deaths_smoothed':0, 'new_vaccinations_smoothed':0}
df_asia.fillna(fill_values, inplace=True)

In [ ]:
# People Vaccinated -- Forward fill by country (start with 0)

# Start with 0
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country)
    mask2 = (df_asia['date'] == (df_asia.loc[mask, 'date'].unique().min()))
    mask_all = mask & mask2
    df_asia.loc[mask_all, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].fillna(0)

# Train Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] < train_end)
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].ffill()
    if df_asia.loc[mask, 'people_vaccinated'].isna().sum() > 0:
        print('train', country)
        
# Validation Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= train_end) & (df_asia['date'] < val_end)
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].ffill()  
    if df_asia.loc[mask, 'people_vaccinated'].isna().sum() > 0:
        print('val', country)
        
# Test Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= val_end)
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].ffill()  
    if df_asia.loc[mask, 'people_vaccinated'].isna().sum() > 0:
        print('test', country)
        

Three options here: 

1. Back fill from known values in Validation + Test Sets 
   - This would avoid any data leakage from the training set into the validation + test sets but not be quite as accurate 

2. Forward fill information from train set into validation + test sets 
    - This would be much more accurate (this is a cumulative value per country and thus be the more accurate approach, but would mean data leakage is occurring. 

3. Drop the variable 

In [ ]:
# Try back filling

# Validation Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= train_end) & (df_asia['date'] < val_end)
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].bfill()  
    if df_asia.loc[mask, 'people_vaccinated'].isna().sum() > 0:
        print('val', country)
        
# Test Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= val_end)
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].bfill()  
    if df_asia.loc[mask, 'people_vaccinated'].isna().sum() > 0:
        print('test', country)

In [ ]:
# That did not work -- rather than dropping the variable we will forward fill completely. 
    # Yes this will mean a bit of data leakage, but it is reasonable to assume that if we knew the previous number of total
    # people vaccinated that this number will be the same moving forward if no new reports/updates made (no new vaccinations)
        
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) 
    df_asia.loc[mask, 'people_vaccinated'] = df_asia.loc[mask, 'people_vaccinated'].ffill()
    
df_asia['people_vaccinated'].isna().sum()

In [ ]:
# stringency_index
df_asia['stringency_index'].isna().sum()

In [ ]:
# Look at when the missing values for stringency_index are
y = []
for date in df_asia['date'].unique():
    y.append(df_asia[df_asia['date'] == date]['stringency_index'].isna().sum())
    
fig, ax = plt.subplots()
ax.plot(df_asia['date'].unique(), y)
ax.set_title('Number of Missing Values in Stringency Index')
ax.set_ylabel('Number of Missing Values')
ax.set_xlabel('Date');

In [ ]:
# Look at average stringency_index by date
y = []
for date in df_asia['date'].unique():
    avg = np.mean(df_asia[df_asia['date'] == date]['stringency_index'])
    y.append(avg)

fig, ax = plt.subplots()
ax.plot(df_asia['date'].unique(), y)
ax.set_title('Average Stringency Index')
ax.set_ylabel('Stringency Index')
ax.set_xlabel('Date');

In [ ]:
# Forward Fill nulls

# Train Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] < train_end)
    df_asia.loc[mask, 'stringency_index'] = df_asia.loc[mask, 'stringency_index'].ffill()
    if df_asia.loc[mask, 'stringency_index'].isna().sum() > 0:
        print('train', country)
        
# Validation Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= train_end) & (df_asia['date'] < val_end)
    df_asia.loc[mask, 'stringency_index'] = df_asia.loc[mask, 'stringency_index'].ffill()  
    if df_asia.loc[mask, 'stringency_index'].isna().sum() > 0:
        print('val', country)
        
# Test Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= val_end)
    df_asia.loc[mask, 'stringency_index'] = df_asia.loc[mask, 'stringency_index'].ffill()  
    if df_asia.loc[mask, 'stringency_index'].isna().sum() > 0:
        print('test', country)

In [ ]:
# So we have three countries where the stringency index was never reported, and, as seen with `people_vaccinated`, 
    # most countries stopped reporting on this value starting in 2023.

# For the countries that never reported we will put the average for the day in -- this value will be averaged in aggregate

for date in df_asia.loc[df_asia['date']<train_end,'date'].unique():
    
    if df_asia[df_asia['date'] == date]['stringency_index'].isna().sum() > 0:
        avg = np.mean(df_asia[df_asia['date'] == date]['stringency_index'])
        
        for country in df_asia[df_asia['stringency_index'].isna()]['location'].unique():
            mask = (df_asia['location'] == country) & (df_asia['date'] == date)
            df_asia.loc[mask, 'stringency_index'].fillna(avg)

For the Validation + test sets again, some options: 

1. Back fill from known values in Validation + Test Sets (although as we saw above this might not work if ALL values are missing for each country. 
2. Forward fill information from train set into validation + test sets 
    - This would be much more accurate (this is a cumulative value per country and thus be the more accurate approach, but would mean data leakage is occurring. 

3. Input 0 for this variable in Validation + Test Sets 
    - This is reasonable only because we know that most countries dropped most if not all of their Covid-19 protections by 2023. 
4. Drop the variable 

In [ ]:
# We will try option 1 and if that fails move to option 3

# Try backfilling:
# Validation Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= train_end) & (df_asia['date'] < val_end)
    df_asia.loc[mask, 'stringency_index'] = df_asia.loc[mask, 'stringency_index'].bfill()  
    if df_asia.loc[mask, 'stringency_index'].isna().sum() > 0:
        print('val', country)
        
# Test Set
for country in df_asia['location'].unique():
    mask = (df_asia['location'] == country) & (df_asia['date'] >= val_end)
    df_asia.loc[mask, 'stringency_index'] = df_asia.loc[mask, 'stringency_index'].bfill()  
    if df_asia.loc[mask, 'stringency_index'].isna().sum() > 0:
        print('test', country)

In [ ]:
# Check if still nulls present
df_asia['stringency_index'].isna().sum()

In [ ]:
# That clearly did not work -- move forward with filling nulls with 0 for 2023 onwards (validation + test sets)
df_asia['stringency_index'] = df_asia['stringency_index'].fillna(0)

In [ ]:
# Sanity Check
df_asia['stringency_index'].isna().sum()

In [ ]:
# Check Total Missing Values
df_asia.isna().sum().sum()

## Aggregate Data + Explore

The next subsection aggregates the data such that there is one row per day.  This will give us an overall average of Asia as a continent and allow us to make general predictions and recommendations on a larger geographical scale. 

The project then goes on and looks at each variable against time, both as the aggregate value and by individual country to further comprehend what data we are working with. 

In [ ]:
# Look at data
df_asia.info()

| Variable | How to aggregate |
|:--|:--|
|`continent` | drop |
| `location` | drop |
| `date` | unique value per day | 
| `new_cases_smoothed` | sum per day |
| `new_deaths_smoothed` | sum per day |
| `people_vaccinated` | sum per day (cumulative value) |
| `new_vaccinations_smoothed` | sum per day |
| `stringency_index` | average per day |

In [ ]:
# Dictionary to create new data frame for time series
asia_ts_data = {}
asia_ts_data['date'] = df_asia['date'].unique()

# Average per day
avg_daily = ['stringency_index']

for var in avg_daily:
    values_to_add = []
    
    for date in df_asia['date'].unique():
        mask = df_asia['date'] == date
        avg = np.mean(df_asia.loc[mask, var])
        values_to_add.append(avg)
        
    asia_ts_data[var] = values_to_add
    
# Sum per day
sum_daily = ['new_cases_smoothed', 'new_deaths_smoothed', 'people_vaccinated', 'new_vaccinations_smoothed']

for var in sum_daily:
    values_to_add = []
    
    for date in df_asia['date'].unique():
        mask = df_asia['date'] == date
        sum_ = np.sum(df_asia.loc[mask, var])
        values_to_add.append(sum_)
        
    asia_ts_data[var] = values_to_add        

In [ ]:
# Create new dataframe
df_asia_ts = pd.DataFrame(asia_ts_data)
df_asia_ts.set_index('date', inplace=True)

In [ ]:
df_asia_ts.loc['2020-12-20':'2021-01-15']

In [ ]:
# Look at new_deaths_smoothed -- our target variable!
fig, ax = plt.subplots()
df_asia_ts['new_deaths_smoothed'].plot(ax=ax)
ax.set_title('New Deaths')
ax.set_ylabel('Number of Deaths');

In [ ]:
# Look at new_deaths_smoothed by country
fig, axes = plt.subplots(ncols=6, nrows=8, figsize=(15,12), sharey=True, sharex=True)

for i,country in enumerate(df_asia['location'].unique()):
    row = i // 6 
    col = i % 6
    ax = axes[row, col]
    
    mask = df_asia['location'] == country
    x = df_asia.loc[mask, 'date'].unique()
    y = df_asia.loc[mask, 'new_deaths_smoothed']
    
    ax.plot(x, y)
    ax.set_title(country)
    ax.set_xticks([])
plt.suptitle('New Deaths by Country by Date', fontsize='x-large');

- The spike in China in January 2023 is due to the end of 'zero COVID' standard.
- The spike in India in June of 2021 is due in part to the Delta Variant that was rampant at the time.
- Even with these known explanations for the spikes we are seeing, they are still notable outliers. 

In [ ]:
# Look at new_cases_smoothed aggregated
df_asia_ts['new_cases_smoothed'].plot();

In [ ]:
# Look at new_cases_smoothed by country

fig, axes = plt.subplots(ncols=6, nrows=8, figsize=(15,15), sharey=True, sharex=True)

for i,country in enumerate(df_asia['location'].unique()):
    row = i // 6 
    col = i % 6
    ax = axes[row, col]
    
    mask = df_asia['location'] == country
    x = df_asia.loc[mask, 'date'].unique()
    y = df_asia.loc[mask, 'new_cases_smoothed']
    
    ax.plot(x, y)
    ax.set_title(country)
    ax.set_xticks([]);

Again, we can attribute this spike seen in 2023 in China to the end of 'Zero-Covid' policies.

In [ ]:
# Look at new_vaccinations_smoothed aggregated
df_asia_ts['new_vaccinations_smoothed'].plot();

In [ ]:
# Look at new_vaccinations smoothed by country

fig, axes = plt.subplots(ncols=6, nrows=8, figsize=(15,15), sharey=True, sharex=True)

for i,country in enumerate(df_asia['location'].unique()):
    row = i // 6 
    col = i % 6
    ax = axes[row, col]
    
    mask = df_asia['location'] == country
    x = df_asia.loc[mask, 'date'].unique()
    y = df_asia.loc[mask, 'new_vaccinations_smoothed']
    
    ax.plot(x, y)
    ax.set_title(country)
    ax.set_xticks([]);

In [ ]:
# Look at people_vaccinated aggreggated
df_asia_ts['people_vaccinated'].plot();

That is an unusual trend at the end there that we would not expect to see as this is a cumulative value -- it should never be smaller than its previous value.  Let's investigate this further and fix.

In [ ]:
# Look at the last 8 days in the dataset
df_asia_ts['people_vaccinated'][-8:]

In [ ]:
# Some countries must not have data for the last 5 days which is causing the averages to produce unexpected 
# Will move forward with using the value found on the 18th as the last value for these 5 days.

list_ = []
list_.append(df_asia_ts['people_vaccinated'][-8])
list_ = list_ * 5
df_asia_ts['people_vaccinated'][-5:] = list_

df_asia_ts['people_vaccinated'][-8:]

In [ ]:
# Look at people_vaccinated by country
fig, axes = plt.subplots(ncols=6, nrows=8, figsize=(15,15), sharey=True, sharex=True)

for i,country in enumerate(df_asia['location'].unique()):
    row = i // 6 
    col = i % 6
    ax = axes[row, col]
    
    mask = df_asia['location'] == country
    x = df_asia.loc[mask, 'date'].unique()
    y = df_asia.loc[mask, 'people_vaccinated']
    
    ax.plot(x, y)
    ax.set_title(country)
    ax.set_xticks([]);

In [ ]:
# Look at stringency_index aggregated
df_asia_ts['stringency_index'].plot();

In [ ]:
# Look at stringency_index by country
fig, axes = plt.subplots(ncols=6, nrows=8, figsize=(15,15), sharey=True, sharex=True)

for i,country in enumerate(df_asia['location'].unique()):
    row = i // 6 
    col = i % 6
    ax = axes[row, col]
    
    mask = df_asia['location'] == country
    x = df_asia.loc[mask, 'date'].unique()
    y = df_asia.loc[mask, 'stringency_index']
    
    ax.plot(x, y)
    ax.set_title(country)
    ax.set_xticks([]);

# Modeling

The project goes through various different models:

1. Naive Model (Baseline)
2. ARIMA (using auto_arima to find best parameters)
3. Multivariate ARIMA
4. Linear Regression
5. Prophet 
6. Multivariate Prophet

All of the above models, aside from Linear Regression, are traditionally used for time series modeling.  The Linear Regression, while not traditionally used for forecasting future date predictions, will be a good comparison.

Each model makes a prediction for the validation set dates, and then is evaluated with the mean absolute error and root mean squared error.  Both of these metrics are commonly used when evaluating time series models.  We will be looking at both metrics as RMSE gives greater punishment to larger errors, which is generally good, however it does make it more sensitive to outliers.  Looking at our data, we clearly have identified some unusual spikes. Thus we will also take a look at the MAE as this more robust to outliers. 

Each model is compared using this error prior to choosing the final model to evaluate with the test set.  We also look at the AIC for the model when applicable.

### Train/Validate/Test Split

In [ ]:
print('Train set will consist of approximately the first {} dates'.format(len(df_asia_ts.index) * .8))
print('Validate set will consist of approximately the next {} dates'.format(len(df_asia_ts.index) * .1))
print('Test set will consist of approximately the last {} dates'.format(len(df_asia_ts.index) * .1))

In [ ]:
# Train Set
first_80_perc = len(df_asia_ts.index) * .8
mask = df_asia_ts.index[:int(first_80_perc)]
train = df_asia_ts.loc[mask]
train.info()

In [ ]:
# Validate Set
val_perc = 139 + int(first_80_perc)
mask = df_asia_ts.index[int(first_80_perc):val_perc]
validate = df_asia_ts.loc[mask]
validate.info()

In [ ]:
# Test Set
mask = df_asia_ts.index[val_perc:]
test = df_asia_ts.loc[mask]
test.info()

## Baseline -- Naive Model

The Naive Method is to predict that the value tomorrow will be the same value that is seen today.  Thus we will shift our data by one for our 'predictions'.  We also look a bit further into our data, looking at the rolling standard deviation and variance of the residuals prior to forecasting  for the validation dates and evaluating the RMSE.

In [ ]:
# The Naive "Model"
baseline = train['new_deaths_smoothed'].shift(1)

# Visualize f
fig, ax = plt.subplots()

train['new_deaths_smoothed'].plot(ax=ax, c='b', label='train data')
baseline.plot(ax=ax, c='r', label='shifted data')
ax.set_title('Naive Predictions - Train')
ax.set_ylabel('Number of Deaths')
ax.set_xlabel('Date')
ax.legend();

In [ ]:
# Train RMSE
baseline_error_train = np.sqrt(mean_squared_error(train['new_deaths_smoothed'][1:], 
                                                  baseline.dropna()))
baseline_error_train

In [ ]:
# look for trends -- want residuals to look like white noise
fig, ax = plt.subplots()

residuals = baseline[1:] - train['new_deaths_smoothed'][1:]
ax.plot(residuals.index, residuals, label='residuals')
ax.plot(residuals.index, residuals.rolling(30).std(), label='rolling std')
ax.set_xticks([])
ax.set_title('Residuals')
ax.legend();

In [ ]:
# check variance
fig, ax = plt.subplots()
ax.plot(residuals.index, residuals.rolling(30).var())
ax.set_xticks([])
ax.set_title('Rolling Variance');

In [ ]:
# "Predict" -- use last known data point for all future predictions
y_preds_baseline = []
y_preds_baseline.append(baseline[-1])
y_preds_baseline = y_preds_baseline * len(validate.index)

# For evaluation, assign y_train and y_val
y_train = train['new_deaths_smoothed']
y_val = validate['new_deaths_smoothed']

In [ ]:
def graph_preds(train_data, validate_data, y_pred, model_name):
    """
    This function takes in the values the model was trained on, and the true validation values, 
    both as pandas series with the date as the index. It also takes in the predictions as a list 
    or pandas series, and graphs all three on a single axis.
    
    Expected input: 
        train_data | pandas series, with datetime as index, of the target training data
        validate_data | pandas series, with datetime as index, of the target validation data
        y_pred | list of the target predictions
        model_name | string
        
    Expected output: matplotlib graph of the target training data, validation data, and predictions 
        plotted against the date
    """

    fig, ax = plt.subplots()

    validate_data.plot(ax=ax, color='b', label='validation data')
    train_data.plot(ax=ax, color='g', label='train data')
    ax.plot(validate_data.index, y_pred, color='r', label='prediction')
    ax.set_title(model_name + ' Predictions')
    ax.set_ylabel('Number of Deaths')
    ax.set_xlabel('Date')
    ax.legend();

In [ ]:
# Visualize
graph_preds(y_train, y_val, y_preds_baseline, 'Naive')

In [ ]:
# Record Evaluation Results

all_results = {'model': [], 'rmse':[], 'mae':[],'train_aic':[]}

def evaluate(y_true, y_pred, all_results, model_name, aic):
    """
    This function takes in a y_true, y_pred, a dictionary called all_results, and a model name. 
    It calculates the RMSE, prints it, and appends it and the model_name to the appropriate dictionary key.
    
    Expected input:
        y_true | true target values as a pandas series or list
        y_pred | target predictions as a pandas series or list
        all_results | python dictionary
        model_name | string
        aic | number or np.nan
        
    Expected output: print out of the updated dictionary, all_results
    """
    # RMSE
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    
    # MAE
    mae = mean_absolute_error(y_true, y_pred)
    
    # Add to dictionary
    all_results['model'].append(model_name)
    all_results['rmse'].append(rmse)
    all_results['mae'].append(mae)
    all_results['train_aic'].append(aic)
    
    print(all_results)

In [ ]:
# evaluate
evaluate(y_val, y_preds_baseline, all_results, 'Naive', np.nan)

## Model 2 - ARIMA

Before jumping into the auto_arima function which performs a grid-search-like function internally to achieve the optimal p, d, and q values, we take a look at the ACF and PACF graphs (for both the original data as well as the once differenced data) to better understand the time series data.  We also visualize the differenced data and perform a Dickey-Fuller test to ascertain whether or not the differenced data is stationary. 

After that manual work, we move forward into utilizing the auto_arima function which does that work for us.  The model is created and then evaluated.

In [ ]:
# ACF
fig, ax = plt.subplots()
plot_acf(train['new_deaths_smoothed'], ax=ax);

In [ ]:
# Take the first difference -- now plot ACF
fig, ax =  plt.subplots()
data = train['new_deaths_smoothed'].diff().dropna()
plot_acf(data, ax=ax);

In [ ]:
# PACF (correlation between current time periord + lags 
    # accounting for the correlation between the intermediate time periods)
fig, ax = plt.subplots()
plot_pacf(train['new_deaths_smoothed'], ax=ax, method='ols-adjusted');

In [ ]:
# PACF of differenced data
fig, ax =  plt.subplots()
data = train['new_deaths_smoothed'].diff().dropna()
plot_pacf(data, ax=ax, method='ols-adjusted');

In [ ]:
# Visualize Differenced Data
fig, ax = plt.subplots()
ax.plot(train['new_deaths_smoothed'].diff().dropna())
ax.set_title('Differenced Data')
ax.set_xticks([]);

In [ ]:
# Check if the once differenced data is stationary

p_val = adfuller(train['new_deaths_smoothed'].diff()[1:])[1]
print(f"The p-value associated with the Dickey-Fuller statistical test is {p_val},")

if p_val < 0.05:
    print("Thus we can safely assume that the differenced data is stationary.")
else:
    print("Thus we cannot reject the null hypothesis that the differenced data is \
not stationary.")

Based on the adjusted Dickey-Fuller test, we can assert that the once differenced data is stationary which means it is suitable for modeling.  However, based on the ACF and PACF visuals, even of the differenced data, we know we will need to include some autoregressive (AR) and moving average (MA) terms.

Let's also take a look at the data once again to see if we should include any seasonality terms in our model.

In [ ]:
# Look for seasonality trends
y_train.plot();

There does not appear to be any seasonality trends present in our data (on the weekly, monthly, nor yearly scale).  We will move forward with excluding these features from our model. 

In [ ]:
# auto_arima function parameters -- values chosen + notes

# time series to train on + predict
y_train = train['new_deaths_smoothed']
# no constants
X_train = train[['new_cases_smoothed', 'people_vaccinated', 'new_vaccinations_smoothed', 'stringency_index']]

# AR term
start_p = 1
max_p = 5 # default

# Order of non-seasonal first-differencing
d=None # default -- calculated
max_d = 2 # default -- from above we know 1 difference is stationary 

# MA term
start_q = 1
max_q = 5 #default

# Seasonality 
seasonal = False 

# Evaluaton to select best ARIMA model
information_criterion = 'aic' # default

# Level of significance
alpha = 0.05 # default

# Scoring default is MSE

In [ ]:
# ARIMA Model
arima = auto_arima(y_train, start_p=start_p, start_q=start_q, seasonal=False, trace=True)

In [ ]:
# Future dates to forecast
n_periods_pred = validate['new_deaths_smoothed'].shape[0]

In [ ]:
# Predict
y_preds_arima = arima.predict(n_periods=n_periods_pred)

# Visualize
graph_preds(y_train, y_val, y_preds_arima, 'ARIMA')

In [ ]:
# AIC
arima_aic = arima.aic()

In [ ]:
# evaluate
evaluate(y_val, y_preds_arima, all_results, 'ARIMA', arima_aic)

We can see that our RMSE and MAE have both improved from our baseline predictions.  Let's see if we can improve that even further.

## Model 3 - Multivariate ARIMA

Rather than just use the target variable, the true 'time series' data of `new_deaths_smoothed`, we will incorporate the other variables that we have available to us. 

In [ ]:
# Multivariate ARIMA model
arima_multi = auto_arima(y_train, X_train, start_p=start_p, start_q=start_q, seasonal=False, trace=True)

In [ ]:
arima_multi.summary()

In [ ]:
# predict
X_val = validate[['new_cases_smoothed', 'people_vaccinated', 'new_vaccinations_smoothed', 'stringency_index']]
y_preds_arima_multi = arima_multi.predict(n_periods=n_periods_pred, X=X_val)

#visualize
graph_preds(y_train, y_val, y_preds_arima_multi, 'Mulltivariate ARIMA')

In [ ]:
#AIC
multi_arima_aic = arima_multi.aic()

# evaluate
evaluate(y_val, y_preds_arima_multi, all_results, 'ARIMA Multivariate', multi_arima_aic)

Again, we see improvement in both RMSE and MAE.

## Model 4 - Linear Regresion

While Linear Regressions are typically not suited for time series data, there is no harm in trying it out.

In [ ]:
# Create + fit the model
lr = OLS(y_train, sm.api.add_constant(X_train))
lr_results = lr.fit()

In [ ]:
# Look at summary of model
print(lr_results.summary())

In [ ]:
# predict
y_preds_lr = lr_results.predict(sm.api.add_constant(X_val))

# Visualize
graph_preds(y_train, y_val, y_preds_lr, 'Linear Regression')

In [ ]:
# Look at the partial regression grid for our variables
fig, ax = plt.subplots(figsize=(10,8))
sm.api.graphics.plot_partregress_grid(lr_results, 
                                      exog_idx=list(X_train.columns.values),
                                      grid=(2,2),
                                      fig=fig)
plt.show()

It is interesting to see here that the relationship between the number of deaths and the number of new vaccinations is positive here.  Intuitively, one would predict that as the total number of people vaccinated increases, the number of deaths caused by the disease would decrease. 

In [ ]:
# AIC
lr_aic = lr_results.aic

# evaluate
evaluate(y_val, y_preds_lr, all_results, 'Linear Regression', lr_aic)

Supprisingly, our Linear Regression model is the most performant model yet!  Let's try Facebook's Prophet last to see if we can improve even further.

## Model 5 - Facebook's Prophet

The prophet library imposes strict conditions for the input column names, so firstly, some reformatting is done. Then the model is created, visualized (the prophet library has some interesting built in visualizations), and evaluated.

In [ ]:
# reformat for Prophet
data_train = {'ds': train.index, 'y': y_train.values}
train_prophet = pd.DataFrame.from_dict(data_train)

data_val = {'ds': validate.index, 'y': y_val.values}
val_prophet = pd.DataFrame.from_dict(data_val)

In [ ]:
# create model - default uncertainty is 80% - update to 95%
prophet_model = Prophet(interval_width=0.95) 

# Fit Model
prophet_model.fit(train_prophet)

In [ ]:
# predict using future dates
y_preds_prophet = prophet_model.predict(val_prophet[['ds']])

In [ ]:
y_preds_prophet.head()

In [ ]:
# Visualize predictions with uncertainty
prophet_model.plot(y_preds_prophet, uncertainty=True)
plt.show()

In [ ]:
# Visualize trends
prophet_model.plot_components(y_preds_prophet)
plt.show()

In [ ]:
# Visualize Predictions (again)
graph_preds(y_train, y_val, y_preds_prophet['yhat'], 'Prophet')

In [ ]:
# evaluate
evaluate(y_val, y_preds_prophet['yhat'], all_results, 'Prophet', np.nan)

Our prophet model is not doing as well as we would like, however this one was only on the time series alone (just the deaths per day data).  Let's create a new prophet model with additional regressors to create a multivariate time series model -- hopefully that will improve our predictions.

## Model 6 - Multivariate Prophet

Again, let's see if we can improve the model by including the other variables in our dataset.

In [ ]:
# format data
train_prophet_multi = pd.concat([train_prophet, X_train.reset_index().drop('date', axis=1)], axis=1)
train_prophet_multi.head()

In [ ]:
# create model
prophet_multi = Prophet(interval_width=0.95)

# Add Regressors for multivariate 
for var in X_train.columns:
    prophet_multi.add_regressor(var, standardize=False)

# Fit Model
prophet_multi.fit(train_prophet_multi)

In [ ]:
# Format Validation Data
val_prophet_multi = pd.concat([val_prophet[['ds']], X_val.reset_index().drop('date', axis=1)], axis=1)
val_prophet_multi.head()

In [ ]:
# predict using future dates
y_preds_prophet_multi = prophet_multi.predict(val_prophet_multi)

y_preds_prophet_multi.head()

In [ ]:
# Visualize predictions with uncertainty
prophet_multi.plot(y_preds_prophet_multi, uncertainty=True)
plt.show()

In [ ]:
# Visualize Trends
prophet_multi.plot_components(y_preds_prophet_multi)
plt.show()

In [ ]:
# Look at regressor coefficients + intervals
regressor_coefficients(prophet_multi)

In [ ]:
# Visualize predictions (again)
graph_preds(y_train, y_val, y_preds_prophet_multi['yhat'], 'Multivariate Prophet')

In [ ]:
# evaluate
evaluate(y_val, y_preds_prophet_multi['yhat'], all_results, 'Prophet Multivariate', np.nan)

## All Model Comparison

In [ ]:
all_results_df = pd.DataFrame.from_dict(all_results)
all_results_df

As we can see that all of our models that included further information, the models that did not rely solely on the number of deaths per day, had better results when forecasting. This is intuitive -- more information available generally leads to a better predictive model. 

We will move forward with the Multivariate Prophet Model as our final model. While the Linear Regression model has a slightly better RMSE, it is forecasting negative values for our test dates which is unreasonable for our situation. The Multivariate Prophet model, more suited to time series, is predicting positive values.  

# Final Model Evaluation

Now that we have chosen our final model we can evaluate it on the hold out test set.

In [ ]:
# format data
train_all = pd.concat([train, validate])

y_train_all = train_all[['new_deaths_smoothed']].reset_index()
y_train_all.rename(columns={'date': 'ds', 'new_deaths_smoothed':'y'}, inplace=True)

X_train_all = train_all[['new_cases_smoothed', 'people_vaccinated', 'new_vaccinations_smoothed', 'stringency_index']]

train_all_prophet = pd.concat([y_train_all,
                               X_train_all.reset_index().drop('date', axis=1)],
                              axis=1)

In [ ]:
# Create final model
prophet_final = Prophet(interval_width=0.95)

# Add Regressors for multivariate
for var in X_train.columns:
    prophet_final.add_regressor(var, standardize=False)
    
# Fit Model
prophet_final.fit(train_all_prophet)

In [ ]:
# Format Test Data
y_test_prophet = test[['new_deaths_smoothed']].reset_index()
y_test_prophet.rename(columns={'date':'ds','new_deaths_smoothed':'y'}, inplace=True)

X_test = test[['new_cases_smoothed', 'people_vaccinated', 'new_vaccinations_smoothed', 'stringency_index']]

test_prophet = pd.concat([y_test_prophet, X_test.reset_index().drop('date', axis=1)], axis=1)
test_prophet.head()

In [ ]:
# Predict
y_preds_final = prophet_final.predict(test_prophet)

In [ ]:
# Visualize predictions with uncertainty
prophet_final.plot(y_preds_final, uncertainty=True)
plt.title('Final Model Predictions')
plt.xlabel('')
plt.ylabel('Number of Deaths')
plt.show()

In [ ]:
# Visualize trends
prophet_final.plot_components(y_preds_final)
plt.show()

In [ ]:
# Look at regressor coefficients + intervals
regressor_coefficients(prophet_final)

In [ ]:
# Visualize predictions
graph_preds(train_all['new_deaths_smoothed'], 
            test['new_deaths_smoothed'], 
            y_preds_final['yhat'],
            'Final Model')

In [ ]:
# RMSE + MAE Final
final_rmse = mean_squared_error(test['new_deaths_smoothed'], y_preds_final['yhat'], squared=False)
final_mae = mean_absolute_error(test['new_deaths_smoothed'], y_preds_final['yhat'])
print('Final RMSE:', round(final_rmse, 2))
print('Final MAE:', round(final_mae, 2))

Our model performed better on the validation data than it did on the test data.  In part, this is because the pandemic had more or less ended by this time (mid 2023), which means the actual values for our data lie very close to zero, whereas our model is predicting the deaths to increase once again (similar to the actuals of July in 2020).  

With an ending root mean squared error of 904 and mean absolute error of only 836, I would say this is a fairly good model, taking into account that this estimation is for the whole continent of Asia rather than one country alone.  

# Conclusion + Recommendations

Overall, the project completes what it set out to do. We have created a predictive time series model that will forecast the number of deaths that will occur in the near future.  Having such a model, especially one that takes into consideration factors such as the stringency index which directly relates to the public health policies currently put in place, can be very useful to public health officials.  With this information they can work on when to ramp up or ease off public health ordinances such as mask mandates and social distancing. 

The recommendations for the business are as follows:


1. **Utilize the model to advise public health officials.**  Once we have forecasting the amount of harm predicted, we can then advise public health officials on what actions to take concerning public health measures such as mask mandates, social distancing, and stay-at-home orders (either to make more strict or lessen).


2. **Utilize the model to aid in resource planning.**  From our forecasts of when spikes will occur, we can make recommendations  to vendors and hospitals concerning resource planning. 


3. **Investigate why deaths tend to be reported higher on Fridays.** From our final model, we can see a weekly trend where reported number of deaths is generally higher on Fridays by about 4 people, and lower earlier in the week by about 3 people. While a slight trend, it would be interesting to investigate why this is;  if it is anything in the hospitals systems or public health trends that can be addressed in such a way that number of deaths lessens.



###  Next Steps

There is always more to do and try!  Below are some ideas for expanding on this project:


- Continue to make data more accurate and complete
    - The missing, purposefully mis-reported, and unusable data all made our model less accurate than it could have been.  Looking for ways in the future to gain accurate, consistent data, as well as ways to utilize the 'constants' would be beneficial in future model endeavors. 
- Data by continent
    - compare predictive models by continent
    - compare how well each continent handled the pandemic
- Data by country
    - create regressive model that can predict number of deaths based on location (and other variables that were in dataset) *Not Time Series*
    - Time series by country and then create a user interface to choose country and date to forecast and then predicts number of deaths